In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader, TensorDataset
from torchvision import datasets
from torchvision import datasets, transforms
from torchvision.transforms import ToTensor

In [3]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download=True,
    transform=ToTensor(),
)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor(),
)

100%|██████████| 26.4M/26.4M [00:07<00:00, 3.49MB/s]
100%|██████████| 29.5k/29.5k [00:00<00:00, 21.2MB/s]
100%|██████████| 4.42M/4.42M [00:01<00:00, 2.33MB/s]
100%|██████████| 5.15k/5.15k [00:00<00:00, 4.96MB/s]


In [4]:
batch_size = 64
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
    print("Shape of X [N, C, H, W]: ", X.shape)
    print("Shape of y: ", y.shape, y.dtype)
    break

Shape of X [N, C, H, W]:  torch.Size([64, 1, 28, 28])
Shape of y:  torch.Size([64]) torch.int64


In [5]:
 # Initialization
device = "cuda" if torch.cuda.is_available() else "cpu"

# Define model
import torch.nn as nn

class NeuralNetwork(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()

        # Defined Layers
        self.fc1 = nn.Linear(28*28, 512)
        self.r1 = nn.ReLU()

        self.fc2 = nn.Linear(512, 512)
        self.r2 = nn.ReLU()

        self.fc3 = nn.Linear(512, 512)
        self.r3 = nn.ReLU()

        self.fc4 = nn.Linear(512, 10)      # Output layer (10 classes)

    def forward(self, x):
        x = self.flatten(x)

        # Layer 1
        x = self.fc1(x)
        x = self.r1(x)

        # Layer 2
        x = self.fc2(x)
        x = self.r2(x)

        # Layer 3
        x = self.fc3(x)
        x = self.r3(x)

        # Layer 4
        logits = self.fc4(x)

        return logits

# Initialize

model = NeuralNetwork().to(device)
print(model)

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (r1): ReLU()
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (r2): ReLU()
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (r3): ReLU()
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)


In [6]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(), lr=1e-3)

In [7]:
def train(dataloader, model, loss_fn, optimizer):
    size = len(dataloader.dataset)
    model.train()
    for batch, (X, y) in enumerate(dataloader):
        X, y = X.to(device), y.to(device)

        # Compute prediction error
        pred = model(X)
        loss = loss_fn(pred, y)

        # Backpropagation
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()                             # Gradient must zero, reset every iteration

        if batch % 100 == 0:
            loss, current = loss.item(), (batch + 1) * len(X)
            print(f"loss: {loss:>7f}  [{current:>5d}/{size:>5d}]")

In [8]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

In [9]:
epochs = 5                        #  One full pass over the dataset
for t in range(epochs):
    print(f"Epoch {t+1}\n-------------------------------")
    train(train_dataloader, model, loss_fn, optimizer)
    test(test_dataloader, model, loss_fn)
print("Done!")

Epoch 1
-------------------------------
loss: 2.304543  [   64/60000]
loss: 2.302107  [ 6464/60000]
loss: 2.293793  [12864/60000]
loss: 2.292279  [19264/60000]
loss: 2.292833  [25664/60000]
loss: 2.285862  [32064/60000]
loss: 2.288369  [38464/60000]
loss: 2.281103  [44864/60000]
loss: 2.279267  [51264/60000]
loss: 2.268898  [57664/60000]
Test Error: 
 Accuracy: 36.1%, Avg loss: 2.271897 

Epoch 2
-------------------------------
loss: 2.275350  [   64/60000]
loss: 2.273571  [ 6464/60000]
loss: 2.259021  [12864/60000]
loss: 2.264725  [19264/60000]
loss: 2.255166  [25664/60000]
loss: 2.241555  [32064/60000]
loss: 2.253069  [38464/60000]
loss: 2.235012  [44864/60000]
loss: 2.238703  [51264/60000]
loss: 2.219010  [57664/60000]
Test Error: 
 Accuracy: 44.8%, Avg loss: 2.221504 

Epoch 3
-------------------------------
loss: 2.230139  [   64/60000]
loss: 2.226245  [ 6464/60000]
loss: 2.198068  [12864/60000]
loss: 2.210814  [19264/60000]
loss: 2.184496  [25664/60000]
loss: 2.156250  [32064/600

In [10]:
torch.save(model.state_dict(), "model.pth")
print("Saved PyTorch Model State to model.pth")

Saved PyTorch Model State to model.pth


In [11]:
torch.save(model,'model.pth')                           # Saves architecture + weights
print("Save Pytorch entire Model State to model.pth")

Save Pytorch entire Model State to model.pth


In [12]:
model = torch.load("model.pth", map_location=device, weights_only=False)
model.eval() 

NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (fc1): Linear(in_features=784, out_features=512, bias=True)
  (r1): ReLU()
  (fc2): Linear(in_features=512, out_features=512, bias=True)
  (r2): ReLU()
  (fc3): Linear(in_features=512, out_features=512, bias=True)
  (r3): ReLU()
  (fc4): Linear(in_features=512, out_features=10, bias=True)
)

In [13]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
